In [ ]:
# Install required libraries
!pip install -q transformers datasets scikit-learn joblib

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
import joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00


In [ ]:
path="/content/drive/MyDrive/Dataset/quotes.csv"
df=pd.read_csv(path)
df.head(5)

,quote,tags
0,"Don't cry because it's over, smile because it ...","life,inspiration,positivity"
1,Be yourself; everyone else is already taken.,"authenticity,humor,inspiration"
2,Two things are infinite: the universe and huma...,"humor,intelligence,philosophy"
3,In three words I can sum up everything I've le...,"life,perseverance,resilience"
4,A room without books is like a body without a ...,"reading,intellect,wisdom"


In [ ]:
# Step 1: Load and preprocess the dataset
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import joblib

# Replace 'path' with your actual file name
data = pd.read_csv(path)  # Make sure 'quotes.csv' has 'quote' and 'tags' columns

# Handle missing or non-string values safely
data['tags'] = data['tags'].fillna('').apply(lambda x: x.split(',') if isinstance(x, str) else [])

# Optionally filter out rows with no tags
data = data[data['tags'].map(len) > 0]

# Fit and transform using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['tags'])

# Save the label encoder
joblib.dump(mlb, 'bert_label_binarizer.pkl')


['bert_label_binarizer.pkl']

In [ ]:
from transformers import BertTokenizer

In [ ]:
from transformers import BertTokenizer  # ✅ Add this import
from torch.utils.data import Dataset, DataLoader
import torch

# === Step 2: Tokenizer and Dataset ===
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class QuoteDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.FloatTensor(self.labels[idx])
        }


In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
from transformers import BertForSequenceClassification  # ✅ Import this
from torch.optim import AdamW
import torch

# Define the model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=y.shape[1],
    problem_type='multi_label_classification'
)

model.train()

# Optimizer setup
optimizer = AdamW(model.parameters(), lr=5e-5)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Assume 'data' and 'y' are already prepared from preprocessing step
X_train, X_test, y_train, y_test = train_test_split(data['quote'], y, test_size=0.2, random_state=42)

# Dataset class (redefine if not already present)
class QuoteDataset(Dataset):
    def __init__(self, quotes, labels):
        self.quotes = quotes
        self.labels = labels

    def __len__(self):
        return len(self.quotes)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.quotes[idx],
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.FloatTensor(self.labels[idx])
        }

# Create dataset and loader
train_dataset = QuoteDataset(X_train.tolist(), y_train)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [ ]:
from tqdm import tqdm  # progress bar

epochs = 3
model.train()

for epoch in range(epochs):
    total_loss = 0
    loop = tqdm(train_loader, leave=True, desc=f"Epoch {epoch+1}")

    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} average loss: {total_loss / len(train_loader):.4f}")


Epoch 1: 100%|██████████| 3/3 [00:41<00:00, 13.70s/it, loss=0.667]


Epoch 1 average loss: 0.6932


Epoch 2: 100%|██████████| 3/3 [00:37<00:00, 12.38s/it, loss=0.604]


Epoch 2 average loss: 0.6226


Epoch 3: 100%|██████████| 3/3 [00:33<00:00, 11.27s/it, loss=0.547]

Epoch 3 average loss: 0.5660


In [ ]:
# Save the model
model.save_pretrained("bert_quote_tag_model")
tokenizer.save_pretrained("bert_quote_tag_model")

('bert_quote_tag_model/tokenizer_config.json',
 'bert_quote_tag_model/special_tokens_map.json',
 'bert_quote_tag_model/vocab.txt',
 'bert_quote_tag_model/added_tokens.json')

In [ ]:
# Step 5: Prediction Function
def predict_tags_bert(quote):
    model = BertForSequenceClassification.from_pretrained("bert_quote_tag_model")
    tokenizer = BertTokenizer.from_pretrained("bert_quote_tag_model")
    mlb = joblib.load('bert_label_binarizer.pkl')
    model.to(device)
    model.eval()

    inputs = tokenizer(quote, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).cpu().numpy()[0]

    threshold = 0.5
    predicted = (probs >= threshold).astype(int)
    return mlb.inverse_transform(np.array([predicted]))[0]

In [ ]:
# === Example usage ===
if __name__ == '__main__':
    test_quote = "Don't cry because it's over, smile because it happened."
    tags = predict_tags_bert(test_quote)
    print("Predicted tags:", tags)


Predicted tags: ('failure', 'reading')
